In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta

# dataset containig information about each trading day

In [3]:
df_prices = pd.read_csv(
        '/content/drive/MyDrive/Mobarakeh.csv',
        parse_dates=['<DTYYYYMMDD>'],  # Ensure Date column is parsed as datetime
        # dayfirst=True,         # Important for Iranian date format (day/month/year)
        encoding='utf-8'       # Handle Persian characters if any
)

In [4]:
df_prices['symbol'] = 'FOLD1'

In [ ]:
df_prices.head()

,<TICKER>,<DTYYYYMMDD>,<FIRST>,<HIGH>,<LOW>,<CLOSE>,<VALUE>,<VOL>,<OPENINT>,<PER>,<OPEN>,<LAST>,symbol
0,S*Mobarakeh Steel,2025-07-02,3007.0,3164.0,3007.0,3070.0,2.770870e+12,902488855,17386,D,3100.0,3133.0,FOLD1
1,S*Mobarakeh Steel,2025-07-01,3100.0,3100.0,3100.0,3100.0,5.503445e+12,1775304991,15736,D,3195.0,3100.0,FOLD1
2,S*Mobarakeh Steel,2025-06-30,3195.0,3195.0,3195.0,3195.0,4.389945e+12,1374004811,9549,D,3293.0,3195.0,FOLD1
3,S*Mobarakeh Steel,2025-06-29,3293.0,3293.0,3293.0,3293.0,1.822477e+11,55343968,1156,D,3394.0,3293.0,FOLD1
4,S*Mobarakeh Steel,2025-06-28,3394.0,3394.0,3394.0,3394.0,1.761190e+11,51891266,774,D,3498.0,3394.0,FOLD1


# Tweets with sentiment Labels

In [5]:
with open('/content/drive/MyDrive/foolad_sentiment_labeled.json', 'r', encoding='utf-8-sig') as f:
    comments_data = json.load(f)


# Convert to DataFrame
df_comments = pd.DataFrame([{
    'id': x['id'],
    'text': x['body'],
    'datetime': x['creation_date_time'],
    'symbol': x['entity_info']['info']['english_trade_symbol'],
    'sentiment': x['sentiment_label'],
    'upvotes': x['upvote_count'],
    'downvotes': x['downvote_count']
} for x in comments_data])

# Convert to datetime (Iran timezone)
df_comments['datetime'] = pd.to_datetime(df_comments['datetime']).dt.tz_convert('Asia/Tehran')

In [ ]:
df_comments.head()

,id,text,datetime,symbol,sentiment,upvotes,downvotes
0,16683616,من منتظرم سهم فولاد بشه 1 ریال تا خرید کنم الب...,2025-06-29 23:55:46+03:30,FOLD1,مثبت,4,1
1,16683465,نخرید کهبدبخت میشید \nحتی اگه مجانیم دادن سهم ...,2025-06-29 23:23:55+03:30,FOLD1,منفی,8,4
2,16683435,بهترین فرصت برای خرید در منفی‌ای فولاد و استفا...,2025-06-29 23:16:56+03:30,FOLD1,مثبت,2,9
3,16683238,حقوقی چه وضیفه ای جز حمایت از سهم دارد \nعج...,2025-06-29 22:42:09+03:30,FOLD1,منفی,8,0
4,16683227,اینجوری پیش بره تا هفته بعد میشه با پول یک بست...,2025-06-29 22:39:09+03:30,FOLD1,مثبت,5,1


In [ ]:
comments_data[5]

{'id': '16682947',
 'body': 'خیلی داغون شدیم',
 'creation_date_time': '2025-06-29T21:43:26+03:30',
 'entity_id': '453',
 'entity_type': 'exchange.asset',
 'entity_info': {'id': '453',
  'type': 'exchange.asset',
  'info': {'trade_symbol': 'فولاد',
   'english_trade_symbol': 'FOLD1',
   'title': 'فولاد مبارکه اصفهان',
   'english_title': 'Mobarakeh Esfehan Steel',
   'short_title': 'فولاد مبارکه اصفهان',
   'english_short_name': 'Mobarakeh Esfehan Steel',
   'slug': 'فولاد_مبارکه_اصفهان'}},
 'account': {'id': '1835662',
  'username': 'Aadolreza49',
  'avatar_id': '',
  'full_name': ''},
 'replies_count': 0,
 'upvote_count': 5,
 'downvote_count': 1,
 'state_id': '1',
 'state': 'updated',
 'sentiment_label': 'منفی'}

In [6]:
def assign_trading_date(dt):
    """Assign comments to trading date (8AM-8AM window)"""
    dt = dt.tz_localize(None)  # Remove timezone for comparison
    if dt.hour >= 8:
        return dt.date()
    else:
        return (dt - timedelta(days=1)).date()

df_comments['trading_date'] = df_comments['datetime'].apply(assign_trading_date)

In [ ]:
df_comments.head()

,id,text,datetime,symbol,sentiment,upvotes,downvotes,trading_date
0,16683616,من منتظرم سهم فولاد بشه 1 ریال تا خرید کنم الب...,2025-06-29 23:55:46+03:30,FOLD1,مثبت,4,1,2025-06-29
1,16683465,نخرید کهبدبخت میشید \nحتی اگه مجانیم دادن سهم ...,2025-06-29 23:23:55+03:30,FOLD1,منفی,8,4,2025-06-29
2,16683435,بهترین فرصت برای خرید در منفی‌ای فولاد و استفا...,2025-06-29 23:16:56+03:30,FOLD1,مثبت,2,9,2025-06-29
3,16683238,حقوقی چه وضیفه ای جز حمایت از سهم دارد \nعج...,2025-06-29 22:42:09+03:30,FOLD1,منفی,8,0,2025-06-29
4,16683227,اینجوری پیش بره تا هفته بعد میشه با پول یک بست...,2025-06-29 22:39:09+03:30,FOLD1,مثبت,5,1,2025-06-29


In [7]:
# Map Persian sentiment to English
sentiment_map = {'منفی': 'negative', 'مثبت': 'positive', 'خنثی': 'neutral'}
df_comments['sentiment'] = df_comments['sentiment'].map(sentiment_map)

# Weighted sentiment score (upvotes - downvotes)
df_comments['sentiment_score'] = np.where(
    df_comments['sentiment'] == 'negative', -1 * (df_comments['upvotes'] + 1),
    np.where(df_comments['sentiment'] == 'positive', df_comments['upvotes'] + 1, 0)
)

# Daily aggregation
daily_sentiment = df_comments.groupby(['trading_date', 'symbol']).agg(
    num_neg=('sentiment', lambda x: (x == 'negative').sum()),
    num_pos=('sentiment', lambda x: (x == 'positive').sum()),
    total=('sentiment', 'count'),
    weighted_score=('sentiment_score', 'sum')
).reset_index()

# Calculate EIst (negative sentiment %)
daily_sentiment['EIst'] = (daily_sentiment['num_neg'] / daily_sentiment['total']) * 100
daily_sentiment['trading_date'] = pd.to_datetime(daily_sentiment['trading_date'])

In [ ]:
df_comments.head(20)

,id,text,datetime,symbol,sentiment,upvotes,downvotes,trading_date,sentiment_score
0,16683616,من منتظرم سهم فولاد بشه 1 ریال تا خرید کنم الب...,2025-06-29 23:55:46+03:30,FOLD1,positive,4,1,2025-06-29,5
1,16683465,نخرید کهبدبخت میشید \nحتی اگه مجانیم دادن سهم ...,2025-06-29 23:23:55+03:30,FOLD1,negative,8,4,2025-06-29,-9
2,16683435,بهترین فرصت برای خرید در منفی‌ای فولاد و استفا...,2025-06-29 23:16:56+03:30,FOLD1,positive,2,9,2025-06-29,3
3,16683238,حقوقی چه وضیفه ای جز حمایت از سهم دارد \nعج...,2025-06-29 22:42:09+03:30,FOLD1,negative,8,0,2025-06-29,-9
4,16683227,اینجوری پیش بره تا هفته بعد میشه با پول یک بست...,2025-06-29 22:39:09+03:30,FOLD1,positive,5,1,2025-06-29,6
5,16682947,خیلی داغون شدیم,2025-06-29 21:43:26+03:30,FOLD1,negative,5,1,2025-06-29,-6
6,16682873,کیه بما اهمیت بده,2025-06-29 21:30:25+03:30,FOLD1,negative,4,0,2025-06-29,-5
7,16682627,به این مقدار قانع نباشید ،هنوز پیشرفت های بزرگ...,2025-06-29 20:57:42+03:30,FOLD1,positive,7,3,2025-06-29,8
8,16682141,با سلام و درود به سهامداران غیور ایران زمین زن...,2025-06-29 19:36:52+03:30,FOLD1,positive,7,20,2025-06-29,8
9,16682023,من نظرم اینه تا خودرو باز نشه بازار منفیه چون ...,2025-06-29 19:22:12+03:30,FOLD1,negative,0,1,2025-06-29,-1


In [ ]:
daily_sentiment.head()

,trading_date,symbol,num_neg,num_pos,total,weighted_score,EIst
0,2025-04-21,FOLD1,107,89,224,75,47.767857
1,2025-04-22,FOLD1,123,52,190,-145,64.736842
2,2025-04-23,FOLD1,93,79,185,122,50.270270
3,2025-04-24,FOLD1,10,14,33,134,30.303030
4,2025-04-25,FOLD1,13,20,34,103,38.235294


In [ ]:
daily_sentiment.shape

(70, 7)

In [ ]:
daily_sentiment.to_csv('output.csv', index=False, encoding='utf-8-sig')

In [8]:
# Calculate next-day return
# df_prices['next_day_return'] = df_prices.groupby('symbol')['close'].pct_change().shift(-1)

# Merge sentiment with prices
df_merged = pd.merge(
    df_prices,
    daily_sentiment,
    left_on=['<DTYYYYMMDD>', 'symbol'],
    right_on=['trading_date', 'symbol'],
    how='left'
)

In [9]:
df_merged.head()

,<TICKER>,<DTYYYYMMDD>,<FIRST>,<HIGH>,<LOW>,<CLOSE>,<VALUE>,<VOL>,<OPENINT>,<PER>,<OPEN>,<LAST>,symbol,trading_date,num_neg,num_pos,total,weighted_score,EIst
0,S*Mobarakeh Steel,2025-07-02,3007.0,3164.0,3007.0,3070.0,2.770870e+12,902488855,17386,D,3100.0,3133.0,FOLD1,NaT,NaN,NaN,NaN,NaN,NaN
1,S*Mobarakeh Steel,2025-07-01,3100.0,3100.0,3100.0,3100.0,5.503445e+12,1775304991,15736,D,3195.0,3100.0,FOLD1,NaT,NaN,NaN,NaN,NaN,NaN
2,S*Mobarakeh Steel,2025-06-30,3195.0,3195.0,3195.0,3195.0,4.389945e+12,1374004811,9549,D,3293.0,3195.0,FOLD1,NaT,NaN,NaN,NaN,NaN,NaN
3,S*Mobarakeh Steel,2025-06-29,3293.0,3293.0,3293.0,3293.0,1.822477e+11,55343968,1156,D,3394.0,3293.0,FOLD1,2025-06-29,98.0,26.0,133.0,-904.0,73.684211
4,S*Mobarakeh Steel,2025-06-28,3394.0,3394.0,3394.0,3394.0,1.761190e+11,51891266,774,D,3498.0,3394.0,FOLD1,2025-06-28,76.0,15.0,100.0,-704.0,76.000000


In [ ]:
# Check missing values in the 'name' column
missing_name_count = df_merged['trading_date'].isna().sum()

print("Missing values in 'name':", missing_name_count)

Missing values in 'name': 4305


In [ ]:
df_merged.shape
df_merged.to_csv('check.csv', index=False, encoding='utf-8-sig')

(4352, 19)

In [ ]:
# Assuming df1 has 'trading_date' and df2 has 'date'
missing_dates = daily_sentiment[~daily_sentiment['trading_date'].isin(df_prices['<DTYYYYMMDD>'])]

# To check if all dates are present
all_present = missing_dates.empty

print("All dates present:", all_present)

if not all_present:
    print("Missing dates:")
    print(missing_dates['trading_date'].unique())


All dates present: False
Missing dates:
<DatetimeArray>
['2025-04-24 00:00:00', '2025-04-25 00:00:00', '2025-05-01 00:00:00',
 '2025-05-02 00:00:00', '2025-05-08 00:00:00', '2025-05-09 00:00:00',
 '2025-05-15 00:00:00', '2025-05-16 00:00:00', '2025-05-22 00:00:00',
 '2025-05-23 00:00:00', '2025-05-29 00:00:00', '2025-05-30 00:00:00',
 '2025-06-04 00:00:00', '2025-06-05 00:00:00', '2025-06-06 00:00:00',
 '2025-06-12 00:00:00', '2025-06-13 00:00:00', '2025-06-14 00:00:00',
 '2025-06-17 00:00:00', '2025-06-19 00:00:00', '2025-06-20 00:00:00',
 '2025-06-26 00:00:00', '2025-06-27 00:00:00']
Length: 23, dtype: datetime64[ns]


# Indicators

In [18]:
# Sort by symbol and date first
final_df = df_merged.sort_values(['symbol', '<DTYYYYMMDD>'])


# 1. Key Metrics
total_shares = 1.5e9  # تعداد سهام
free_float = 0.36     # سهام شناور
floor_liquidity = 39.088e6  # حجم مبنا
EPS = 591

# Volatility of Closing Price (VCP) - Direction Only
final_df['VCP_dir'] = np.where(final_df.groupby('symbol')['<CLOSE>'].diff() >= 0, 'up', 'down')

# Volatility of Opening Price (VOP) - Direction Only
final_df['VOP_dir'] = np.where(final_df.groupby('symbol')['<OPEN>'].diff() >= 0, 'up', 'down')

# Daily Price Change Magnitude (for reference)
final_df['daily_change'] = final_df['<CLOSE>'] - final_df['<OPEN>']

# Turnover Rate (TR) Classification
# df['TR_class'] = np.where(df['volume']/df['shares_outstanding'] >= 0.3, 'high', 'low')

# Turnover Rate (TR) - Simplified version
final_df['TR_Simplified'] = final_df['<VOL>'] / final_df['<VOL>'].rolling(30).mean()

final_df['TR'] = final_df['<VOL>'] / 1.5e9  # 1.5B shares outstanding
final_df['TR_Class'] = np.where(final_df['TR'] >= 0.3, 'High', 'Low')

final_df['EPS'] = 591  # Current annual EPS (1403)
final_df['PE'] = final_df['<CLOSE>'] / 591

final_df['VPE_dir'] = np.where(final_df.groupby('symbol')['PE'].diff() >= 0, 'up', 'down')

# final_df['PB'] = final_df['<CLOSE>'] / (final_df['Total_Book_Value'] / total_shares)
# final_df['VPB'] = final_df.groupby('symbol')['PB'].diff()  # This is the key VPB calculation
# final_df['VPB_dir'] = np.where(final_df['VPB'] >= 0, 'up', 'down')

# OR use Sector Average P/B if book value is unavailable
sector_avg_pb = 1.79   # From earlier data
final_df['PB'] = final_df['<CLOSE>'] / (final_df['<CLOSE>'].mean() / sector_avg_pb)  # Proxy calculation

final_df['Is_Stable'] = final_df['<VOL>'] >= 39.088e6

final_df['FF_TR'] = final_df['<VOL>'] / (1.5e9 * 0.36)  # Free-float adjusted
final_df['FF_TR_Class'] = np.where(final_df['FF_TR'] >= 0.3, 'High', 'Low')



In [23]:
df_cleaned = final_df.dropna(subset=['trading_date'])

In [25]:
# Fixed classification
conditions = [
    (df_cleaned['EIst'] <= 50),
    (df_cleaned['EIst'] > 50) & (df_cleaned['EIst'] <= 60),
    (df_cleaned['EIst'] > 60)
]
choices = ['non_negative', 'weak_negative', 'strong_negative']

# Key fix: Add default='neutral' to ensure string dtype
df_cleaned['EIst_class'] = np.select(conditions, choices, default='neutral')

print(df_cleaned)

             <TICKER> <DTYYYYMMDD>  <FIRST>  <HIGH>   <LOW>  <CLOSE>  \
49  S*Mobarakeh Steel   2025-04-21   4479.0  4481.0  4280.0   4438.0   
48  S*Mobarakeh Steel   2025-04-22   4340.0  4360.0  4305.0   4310.0   
47  S*Mobarakeh Steel   2025-04-23   4330.0  4439.0  4200.0   4389.0   
46  S*Mobarakeh Steel   2025-04-26   4520.0  4520.0  4300.0   4435.0   
45  S*Mobarakeh Steel   2025-04-27   4302.0  4330.0  4302.0   4303.0   
44  S*Mobarakeh Steel   2025-04-28   4289.0  4391.0  4174.0   4205.0   
43  S*Mobarakeh Steel   2025-04-29   4227.0  4324.0  4175.0   4254.0   
42  S*Mobarakeh Steel   2025-04-30   4250.0  4313.0  4224.0   4271.0   
41  S*Mobarakeh Steel   2025-05-03   4143.0  4144.0  4143.0   4143.0   
40  S*Mobarakeh Steel   2025-05-04   4019.0  4098.0  4019.0   4034.0   
39  S*Mobarakeh Steel   2025-05-05   4050.0  4118.0  3959.0   4019.0   
38  S*Mobarakeh Steel   2025-05-06   4035.0  4085.0  3930.0   3989.0   
37  S*Mobarakeh Steel   2025-05-07   4098.0  4108.0  4027.0   40

/tmp/ipython-input-25-1323599328.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['EIst_class'] = np.select(conditions, choices, default='neutral')


In [26]:
df_cleaned.head()

,<TICKER>,<DTYYYYMMDD>,<FIRST>,<HIGH>,<LOW>,<CLOSE>,<VALUE>,<VOL>,<OPENINT>,<PER>,...,TR,TR_Class,EPS,PE,VPE_dir,PB,Is_Stable,FF_TR,FF_TR_Class,EIst_class
49,S*Mobarakeh Steel,2025-04-21,4479.0,4481.0,4280.0,4438.0,1.113804e+13,2509813619,52958,D,...,1.673209,High,591,7.509306,up,1.768076,True,4.647803,High,non_negative
48,S*Mobarakeh Steel,2025-04-22,4340.0,4360.0,4305.0,4310.0,4.885073e+12,1133517816,26178,D,...,0.755679,High,591,7.292724,down,1.717081,True,2.099107,High,strong_negative
47,S*Mobarakeh Steel,2025-04-23,4330.0,4439.0,4200.0,4389.0,9.397860e+12,2141175113,36567,D,...,1.427450,High,591,7.426396,up,1.748555,True,3.965139,High,weak_negative
46,S*Mobarakeh Steel,2025-04-26,4520.0,4520.0,4300.0,4435.0,5.455518e+12,1229973497,27072,D,...,0.819982,High,591,7.504230,up,1.766881,True,2.277729,High,weak_negative
45,S*Mobarakeh Steel,2025-04-27,4302.0,4330.0,4302.0,4303.0,3.570018e+12,829568733,18190,D,...,0.553046,High,591,7.280880,down,1.714293,True,1.536238,High,strong_negative


In [28]:
df_cleaned.to_csv('/content/drive/MyDrive/StockIndicators.csv', index=False, encoding='utf-8-sig')